# GDELT Country Exploration: Maritime Port Disruption Analysis

## Project Overview

This notebook demonstrates a comprehensive data analytics approach to predict maritime port disruptions using the GDELT (Global Database of Events, Language, and Tone) dataset. The analysis focuses on the **Transpacific Route** and key ports across Canada, USA, China, and Japan.

### Key Objectives:
1. **Data Validation & Cleaning**: Validate and clean raw GDELT data to ensure quality and reliability
2. **Theme Identification**: Identify port-related news with relevant themes (infrastructure, trade, incidents)
3. **Sentiment Analysis**: Filter emotionally-charged news to focus on factual reporting
4. **Predictive Analytics**: Analyze news patterns and growth to predict potential disruptions

### Workflow:
```
Raw Data → Cleaning & Validation → Theme Extraction → Sentiment Filtering → Pattern Analysis → Predictions
```

---

## 1. Setup: Libraries and Dependencies

Import necessary libraries for data processing, transformation, and analysis using PySpark SQL functions.

In [ ]:
# ============================================
# LIBRARY IMPORTS
# ============================================
# Core PySpark libraries for distributed data processing

from pyspark.sql.functions import *  # SQL functions for data transformation
import datetime                       # Date/time operations
from pyspark.sql.window import Window # Window functions for time-series analysis

## 2. User-Defined Functions (UDFs)

Custom functions created to support geospatial calculations and data transformations throughout the analysis.

In [ ]:
# ============================================
# UDF: GEOSPATIAL DISTANCE CALCULATION
# ============================================

def cal_lat_log_dist(df, lat1, long1, lat2, long2):
    """
    Calculate the great-circle distance between two geographic points using the Haversine formula.
    
    This function computes the shortest distance over the earth's surface between two points
    specified by their latitude and longitude coordinates.
    
    Parameters:
    -----------
    df : DataFrame
        Input PySpark DataFrame to which the distance column will be added
    lat1 : str
        Column name containing latitude of the first point (in degrees)
    long1 : str
        Column name containing longitude of the first point (in degrees)
    lat2 : str
        Column name containing latitude of the second point (in degrees)
    long2 : str
        Column name containing longitude of the second point (in degrees)
    
    Returns:
    --------
    DataFrame
        Original DataFrame with an additional 'distance_in_kms' column containing
        the calculated distance in kilometers, rounded to 4 decimal places
    
    Formula:
    --------
    Uses the Haversine formula:
    distance = R × arccos(sin(lat1) × sin(lat2) + cos(lat1) × cos(lat2) × cos(long1 - long2))
    where R = 6371 km (Earth's radius)
    
    Example:
    --------
    >>> df_with_distance = cal_lat_log_dist(df, 'port_lat', 'port_long', 'event_lat', 'event_long')
    """
    
    # Haversine formula implementation using PySpark SQL functions
    df = df.withColumn('distance_in_kms',
        round(
            (acos(
                (sin(radians(col(lat1))) * sin(radians(col(lat2)))) +
                ((cos(radians(col(lat1))) * cos(radians(col(lat2)))) *
                 (cos(radians(long1) - radians(long2))))
            ) * lit(6371.0)),  # Earth's radius in kilometers
            4  # Round to 4 decimal places for precision
        )
    )
    return df

## 3. Data Loading from Bronze Layer

### Medallion Architecture - Bronze Layer
The Bronze layer contains raw, unprocessed data from source systems. This follows the **Medallion Architecture** best practice for data lakes:
- **Bronze**: Raw ingestion layer (current layer)
- **Silver**: Cleaned and validated data
- **Gold**: Business-level aggregates

### Data Sources:

1. **GDELT_EVENTS**: Core GDELT events table containing global news events with actors, locations, and event codes
2. **GDELT_GKG** (Global Knowledge Graph): Enhanced metadata including themes, tone, and sentiment analysis
3. **PORTS_DICTIONARY**: Reference table with global port locations and coordinates
4. **CAMEO_DICTIONARY**: CAMEO (Conflict and Mediation Event Observations) event code descriptions for event classification

All tables are loaded directly from the Bronze database in their raw form for initial exploration and validation.

In [ ]:
# ============================================
# LOAD RAW DATA FROM BRONZE LAYER
# ============================================

# Load GDELT events table - contains structured event data
GDELT_EVENTS = spark.sql("SELECT * FROM BRONZE.GDELT_EVENTS")

# Load port locations reference data - contains coordinates and metadata for ports worldwide
PORT_LOCATIONS_DIM = spark.sql("SELECT * FROM BRONZE.PORTS_DICTIONARY")

# Load CAMEO event code dictionary - maps event codes to human-readable descriptions
CAMEO_DICTIONARY = spark.sql("SELECT * FROM BRONZE.CAMEO_DICTIONARY")

# Load GDELT Global Knowledge Graph - contains themes, tones, and enriched metadata
GKG = spark.sql("SELECT * FROM BRONZE.GDELT_GKG")

## 4. Data Quality: Cleaning PORT_LOCATIONS_DIM

### Data Quality Issues Identified:
The raw port locations data requires cleaning due to several data quality issues:
- ❌ Missing coordinate values (NULL latitudes/longitudes)
- ❌ Inconsistent formatting with embedded spaces
- ❌ Directional indicators (N, S, E, W) included in numeric values
- ❌ Lack of standardized decimal coordinate format

### Cleaning Process:

1. **Filter Missing Values**: Remove records with NULL coordinates
2. **Standardize Format**: Remove whitespace from coordinate strings
3. **Extract Orientation**: Parse directional indicators (N/S/E/W)
4. **Convert Coordinates**: Transform to decimal degrees with proper signs:
   - North (N) and East (E) → Positive values
   - South (S) and West (W) → Negative values
5. **Select Clean Columns**: Output only validated, standardized fields

### Output Schema:
| Column | Type | Description |
|--------|------|-------------|
| COUNTRY | String | Country name |
| PORT | String | Port name |
| LATITUDE_CORRECTED | Float | Decimal latitude (-90 to 90) |
| LONGITUDE_CORRECTED | Float | Decimal longitude (-180 to 180) |

In [ ]:
# ============================================
# PORT LOCATIONS DATA CLEANING PIPELINE
# ============================================

PORT_LOCATIONS_DIM_CLEANED = (
    PORT_LOCATIONS_DIM
    
    # STEP 1: Filter out records with missing coordinate data
    .filter("LATITUDE IS NOT NULL")   # Ensure latitude exists
    .filter("LONGITUDE IS NOT NULL")  # Ensure longitude exists
    
    # STEP 2: Standardize format - remove embedded whitespace
    .withColumn("LATITUDE", regexp_replace(col("LATITUDE"), " ", ""))   # Clean latitude string
    .withColumn("LONGITUDE", regexp_replace(col("LONGITUDE"), " ", "")) # Clean longitude string
    
    # STEP 3: Extract directional orientation from coordinate strings
    # Format example: "37.7749N" → Extract "N"
    .withColumn("Lat_Ori", substring(col("LATITUDE"), -1, 1))   # Get last char (N/S)
    .withColumn("Long_Ori", substring(col("LONGITUDE"), -1, 1)) # Get last char (E/W)
    
    # STEP 4: Convert to decimal degrees with correct signs
    # Apply conversion logic based on directional indicators
    .withColumn("LATITUDE_CORRECTED",
        when(col("Lat_Ori") == 'S',  # South → Negative
             expr("substring(LATITUDE, 1, length(LATITUDE) - 1)") * -1)
        .when(col("Lat_Ori") == 'N',  # North → Positive
              expr("substring(LATITUDE, 1, length(LATITUDE) - 1)"))
        .when(col("Lat_Ori") == 'E',  # Edge case: E marked as lat
              expr("substring(LATITUDE, 1, length(LATITUDE) - 1)") * -1)
        .otherwise(999.999)  # Flag for unmapped values (data quality check)
    )
    .withColumn("LONGITUDE_CORRECTED",
        when(col("Long_Ori") == 'E',  # East → Positive
             expr("substring(LONGITUDE, 1, length(LONGITUDE) - 1)"))
        .when(col("Long_Ori") == 'W',  # West → Negative
              expr("substring(LONGITUDE, 1, length(LONGITUDE) - 1)") * -1)
        .when(col("Lat_Ori") == 'N',  # Edge case: N marked as long
              expr("substring(LATITUDE, 1, length(LATITUDE) - 1)") * -1)
        .otherwise(999.999)  # Flag for unmapped values (data quality check)
    )
    
    # STEP 5: Select only clean, relevant columns for downstream processing
    .select("COUNTRY", "PORT", "LATITUDE_CORRECTED", "LONGITUDE_CORRECTED")
)

# Display sample of cleaned data for validation
# PORT_LOCATIONS_DIM_CLEANED.display()

## OBJECTIVE - DATA ANALYSIS:

To start doing some analysis with our data. We have to do a clear explanaition of our objective. Our Objective is:

**Detect and Precidit possible complications in port related to the Transpacific Route**

First, we need to identify the news that could be related to Ports inside the Transpacific Route. The countries we select to analyze are: Canada, United States, China and Japan. and for each country we select the three more importants ports that are useden in the Transpacific Route. The following list is a compilation per country that discribes: 
* Name of Country
* Code for Country
* Name of Port
* Location of the Port
* Code of the Location of the Port

**For further reference in locations you can check the following summary**

**CANADA**
Code for country = 'CA'

1. Port of Vancouver - British Columbia (CA02)
2. Puerto de Prince Rupert - Columbia Británica (CA02)
3. Port of Montreal - Quebec (CA10)

**USA**
Code for country = 'US'

1. Port of Los Angeles - California (USCA)
2. Port of Long Beach - California (USCA)
3. Port of Oakland - California (USCA)

**CHINA**
Code for country = 'CH'

1. Port of Shanghai - Shanghai (CH23)
2. Port of Shenzhen - Guangdong Province (CH30)
3. Port of Ningbo-Zhoushan - Zhejiang Province (CH02)

**JAPAN**
Code for Country = 'JA'

1. Port of Tokyo - Tokyo (JA40)
2. Port of Yokohama - Kanagawa Prefecture (JA19)
3. Port of Nagoya - Aichi Prefecture (JA01)

## 5. GKG Data Cleaning & Preparation

### Why Data Cleaning is Critical:
The GDELT GKG (Global Knowledge Graph) table contains complex, semi-structured data that requires extensive cleaning before analysis. Raw GKG data presents several challenges:
- **Inconsistent date formats**: Need standardization for time-series analysis
- **Delimited fields**: LOCATIONS and TONE columns contain multiple values separated by delimiters
- **Large volume**: Filtering by time period reduces computational overhead
- **Geographic specificity**: Need to extract country and location codes for filtering

### Cleaning Operations:

| Step | Operation | Purpose |
|------|-----------|---------|
| 1 | Date Standardization | Convert YYYYMMDD string → proper Date type |
| 2 | Time Period Filter | Limit to 2022-01-01 to 2024-07-31 for relevant analysis |
| 3 | Location Parsing | Split LOCATIONS field (delimited by '#') to extract codes |
| 4 | Tone Decomposition | Split TONE field (delimited by ',') into sentiment components |
| 5 | Geographic Filter | Keep only news from Transpacific Route port locations |

### Extracted Sentiment Metrics:
- **AverageTone**: Overall sentiment score (-100 to +100, negative = bad news)
- **TonePositiveScore**: Percentage of positive words in the document
- **ToneNegativeScore**: Percentage of negative words in the document
- **Polarity**: Spread between positive and negative (higher = more extreme)

In [ ]:
# ============================================
# GKG PRIMARY DATA CLEANING PIPELINE
# ============================================

GKG_PRINCIPAL_CLEANING = (
    GKG
    
    # STEP 1: Standardize date format for time-series analysis
    .withColumn("Date", to_date(col("DATE"), "yyyyMMdd"))  # Convert YYYYMMDD string to Date type
    
    # STEP 2: Filter to relevant time period (Jan 2022 - July 2024)
    # This reduces data volume and focuses on recent, relevant events
    .filter("Date >= '2022-01-01' and Date < '2024-08-01'")
    
    # STEP 3: Parse location information from delimited LOCATIONS field
    # Format: "type#typecode#countrycode#locationcode#..."
    .withColumn("CountryCode", split(col("LOCATIONS"), "#").getItem(2))   # Extract country code (index 2)
    .withColumn("LocationCode", split(col("LOCATIONS"), "#").getItem(3))  # Extract location code (index 3)
    
    # STEP 4: Decompose TONE field into sentiment components
    # Format: "averageTone,positiveScore,negativeScore,polarity,..."
    .withColumn("AverageTone", split(col("TONE"), ",").getItem(0))        # Overall sentiment (-100 to +100)
    .withColumn("TonePositiveScore", split(col("TONE"), ",").getItem(1))  # % positive words
    .withColumn("ToneNegativeScore", split(col("TONE"), ",").getItem(2))  # % negative words
    .withColumn("Polarity", split(col("TONE"), ",").getItem(3))           # Emotional spread
    
    # STEP 5: Filter to Transpacific Route port locations only
    # Location codes correspond to major ports in Canada, USA, China, and Japan
    # See "Objective - Data Analysis" section for detailed location code mappings
    .filter(col("LocationCode").isin(
        'CA02',  # British Columbia, Canada (Vancouver, Prince Rupert)
        'CA10',  # Quebec, Canada (Montreal)
        'USCA',  # California, USA (Los Angeles, Long Beach, Oakland)
        'CH23',  # Shanghai, China
        'CH30',  # Guangdong, China (Shenzhen)
        'CH02',  # Zhejiang, China (Ningbo-Zhoushan)
        'JA40',  # Tokyo, Japan
        'JA19',  # Kanagawa, Japan (Yokohama)
        'JA01'   # Aichi, Japan (Nagoya)
    ))
)

# The cleaned dataset is now ready for sentiment filtering and theme analysis

In [0]:
TABLE_OF_DATES = (
GKG_PRINCIPAL_CLEANING
.select("Date").distinct()
)

## 6. Filtering Emotionally-Charged News

### The Problem: Emotional Bias in News Reporting

As a logistics company making critical operational decisions, we cannot rely on emotionally-charged reporting that may distort the factual situation. Highly emotional news can lead to:
- **Misinterpretation** of actual risk levels
- **Overreaction** to minor incidents
- **Poor decision-making** based on sensationalism rather than facts

### GDELT's Emotional Charge Identification

According to the GDELT documentation, emotionally-charged news exhibits a specific pattern:
- **Neutral average tone** (close to 0) → Not clearly positive or negative
- **High polarity** (large spread) → Contains both very positive and very negative language

This combination indicates reporting that uses extreme language from both perspectives, suggesting emotional manipulation rather than objective journalism.

### Our Filtering Criteria:

| Metric | Threshold | Rationale |
|--------|-----------|-----------|
| **Neutrality Range** | -0.5 to +0.5 | Tone is neither clearly positive nor negative |
| **High Polarity** | ≥ 9.0 | Above 85th percentile based on data distribution |

**Filtering Logic:**
```
Emotionally Charged = (AverageTone between -0.5 and 0.5) AND (Polarity ≥ 9)
Keep Only: News WHERE Emotionally Charged = False
```

### Why This Matters:
By filtering out emotional content, we ensure our disruption predictions are based on **factual reporting** of actual incidents rather than sensationalized coverage.

In [ ]:
# ============================================
# FILTER EMOTIONALLY-CHARGED NEWS
# ============================================

GKG_NOT_EMOTIONAL_CHARGE = (
    GKG_PRINCIPAL_CLEANING
    
    # STEP 1: Identify neutral tone news
    # News with AverageTone close to 0 (-0.5 to +0.5) are considered neutral
    .withColumn("Neutrality",
        when((col("AverageTone") >= -0.5) & (col("AverageTone") <= 0.5), 1)  # Flag as neutral (1)
        .otherwise(0)  # Not neutral (0)
    )
    
    # STEP 2: Flag emotionally-charged content
    # Combination of neutral tone + high polarity = emotional manipulation
    .withColumn("EC",  # EC = Emotional Charge flag
        when(
            (col("Neutrality") == 1) &      # Tone is neutral, BUT...
            (col('Polarity') >= 9),         # ...polarity is extreme (≥85th percentile)
            1  # Flag as emotionally charged
        )
        .otherwise(0)  # Not emotionally charged
    )
    
    # STEP 3: Filter to keep only factual, non-emotional news
    # We want EC == 0 (not emotionally charged) for objective analysis
    .filter("EC == 0")
)

# Result: Dataset now contains only news with objective, factual reporting
# This ensures our disruption predictions are based on real events, not sensationalism

---

## 7. Approach 1: Theme-Based News Growth Analysis

### Objective:
Predict port disruptions by analyzing **growth patterns** in negative news coverage across multiple relevant themes.

### Hypothesis:
An **increase in negative news** related to port activities and associated themes (infrastructure, trade, incidents) may signal impending disruptions before they become critical.

### Theme Selection Strategy:

We identify port-related news through a **base filter** plus **key theme combinations**:

| Theme | GDELT Code | Why It Matters |
|-------|------------|----------------|
| **Base Filters** | | |
| PORT | `THEMES LIKE '%PORT%'` | Direct port mentions |
| TRANSPORT | `THEMES LIKE '%TRANSPORT%'` | Shipping and logistics |
| NOT AIRPORT | `THEMES NOT LIKE '%AIRPORT%'` | Exclude air transport |
| | | |
| **Key Themes** | | |
| Transport Infrastructure | `TRANSPORT_INFRASTRUCTURE` | Port facility issues, construction |
| Trade | `TRADE` | Trade disputes, tariffs, policy changes |
| Macroeconomic | `MACROECONOMIC` | Economic factors affecting ports |
| Public Sector | `PUBLIC_SECTOR` | Government regulations, strikes |
| Maritime Incident | `MARITIME_INCIDENT` | Accidents, blockages, safety issues |

### Analysis Method:
1. **Identify** news matching base criteria (PORT + TRANSPORT - AIRPORT)
2. **Tag** each news article with applicable themes (1 if present, 0 if not)
3. **Count** total theme occurrences per article
4. **Filter** to negative sentiment only (AverageTone < 0)
5. **Aggregate** daily counts by location and theme combination
6. **Analyze growth** patterns over time using window functions

In [ ]:
# ============================================
# APPROACH 1: THEME-BASED NEWS GROWTH ANALYSIS
# ============================================

# PART 1: Theme identification and aggregation
GKG_PORTS_OV_FIRST_APPROACH = (        
    GKG_NOT_EMOTIONAL_CHARGE
    
    # STEP 1: Apply base filters to identify port-related news
    .withColumn("BaseNews",
        when(
            (col("THEMES").like("%PORT%")) &          # Must mention ports
            (col("THEMES").like("%TRANSPORT%")) &     # Must mention transport
            (~col("THEMES").like("%AIRPORT%")),       # Exclude airports
            1  # Flag as base port news
        )
    )
    .filter("BaseNews == 1")  # Keep only port-related news
    
    # STEP 2: Tag news with relevant theme flags (1 if present, 0 if not)
    .withColumn("NewsWithTINFA",  # Transport Infrastructure
        when(col("THEMES").like("%TRANSPORT_INFRASTRUCTURE%"), 1))
    .withColumn("NewsWithTRADE",  # Trade-related
        when(col("THEMES").like("%TRADE%"), 1))
    .withColumn("NewsWithME",     # Macroeconomic
        when(col("THEMES").like("%MACROECONOMIC%"), 1))
    .withColumn("NewsWithPS",     # Public Sector
        when(col("THEMES").like("%PUBLIC_SECTOR%"), 1))
    .withColumn("NewsWithMI",     # Maritime Incident
        when(col("THEMES").like("%MARITIME_INCIDENT%"), 1))
    .fillna(0)  # Fill NULL flags with 0
    
    # STEP 3: Calculate total number of themes present in each news article
    .withColumn("Total",
        col("NewsWithTINFA") + col("NewsWithTRADE") + 
        col("NewsWithME") + col("NewsWithPS") + col("NewsWithMI")
    )
    
    # STEP 4: Filter to negative news only (potential disruption signals)
    .filter("AverageTone < 0")  # Only keep news with negative sentiment
    
    # STEP 5: Aggregate by date, theme combination, and location
    .groupby("Date", "Total", "LocationCode")
    .agg(
        sum("NewsWithTINFA").alias("NewsWithTINFA"),     # Count infrastructure news
        sum("NewsWithME").alias("NewsWithME"),           # Count macro news
        sum("NewsWithPS").alias("NewsWithPS"),           # Count public sector news
        sum("NewsWithMI").alias("NewsWithMI"),           # Count incident news
        sum("Total").alias("TotalThemesFindings"),       # Total theme occurrences
        count("Total").alias("NumberOfNews")             # Total news count
    )
)

# PART 2: Create complete date × location × theme combination grid
# This ensures we have records for all dates, even with zero news (important for time-series)
GKG_PORTS_OV_FIRST_APPROACH_CD = (
    TABLE_OF_DATES  # All dates in our analysis period
    .crossJoin(  # Cartesian product with all possible combinations
        GKG_PORTS_OV_FIRST_APPROACH
        .select("Total", "LocationCode")
        .distinct()
    )
)

# PART 3: Left join to fill in missing date/location/theme combinations with zeros
GKG_PORTS_FIRST_APPROACH = (
    GKG_PORTS_OV_FIRST_APPROACH_CD
    .join(GKG_PORTS_OV_FIRST_APPROACH, ["Date", "Total", "LocationCode"], "left")
    .fillna(0)  # Fill missing combinations with 0 (no news that day)
    .persist()  # Cache for repeated use in visualizations
)

# Materialize the dataframe to validate row count
GKG_PORTS_FIRST_APPROACH.count()

# NOTE: Window functions for growth analysis applied in visualization cells below
# Example: lag(col("NumberOfNews"), 1).over(Window.partitionBy("Total").orderBy("Date"))

In [0]:
display(GKG_PORTS_FIRST_APPROACH)

In [0]:
display(
GKG_PRINCIPAL_CLEANING
.withColumn("BaseNews", when((col("THEMES").like("%PORT%")) & (col("THEMES").like("%TRANSPORT%")) & (~col("THEMES").like("%AIRPORT%")),1)) # BASE FLAG NEWS THAT HAVE THEME PORT AND TRANSPORTATIONS AND NOT AIRPORTS
.filter("BaseNews == 1") # SELECT NEWS ONLY RELATED TO BASE NEWS FLAG
.filter("LocationCode == 'CA02'")
.filter("Date == '2024-07-23'")
)

In [0]:
display(
GKG_PORTS_FIRST_APPROACH
.withColumn("NumberOfNewsPastDay", lag(col("NumberOfNews"),1).over(Window.partitionBy("Total","LocationCode").orderBy("Date")))
.withColumn("GrowthNumberOfNewsPastDay", (col("NumberOfNews") - col("NumberOfNewsPastDay"))/col("NumberOfNewsPastDay"))
.filter("LocationCode == 'CA02'")
.withColumn("LagGrowthNumberOfNewsPastDay", lag(col("GrowthNumberOfNewsPastDay"),1).over(Window.partitionBy("Total","LocationCode").orderBy("Date")))
.fillna(0)
)

Databricks visualization. Run in Databricks to view.

---

## 8. Approach 2: Weighted News Scoring System

### Objective:
Assign **weighted scores** to news based on the number of critical themes present, emphasizing articles that mention multiple risk factors.

### Hypothesis:
News articles mentioning **multiple relevant themes** are more significant indicators of potential disruptions than single-theme articles. A weighted scoring system helps prioritize monitoring efforts.

### Weighting Strategy:

The more themes an article contains, the higher its importance score:

| Themes Present | Weight Multiplier | Rationale |
|----------------|-------------------|-----------|
| **5 themes** | 500× | Extremely rare, likely major incident |
| **4 themes** | 250× | Significant multi-faceted issue |
| **3 themes** | 100× | Complex situation affecting multiple areas |
| **2 themes** | 5× | Moderate concern with multiple factors |
| **1 theme** | 0× | Excluded (insufficient signal) |

### Why This Approach?

**Advantages:**
- **Signal amplification**: Multi-theme news gets exponentially higher weight
- **Noise reduction**: Single-theme news filtered out as potential noise
- **Prioritization**: Helps identify the most critical news to monitor

**Use Case:**
Logistics operators can focus attention on high-weighted-score days/locations, indicating complex, multi-factor disruption risks.

### Analysis Method:
1. Apply same theme identification as Approach 1
2. Calculate weighted score based on theme count
3. Aggregate weighted scores by date and location  
4. Compare with lagged values to detect spikes

In [ ]:
# ============================================
# APPROACH 2: WEIGHTED NEWS SCORING SYSTEM
# ============================================

# PART 1: Theme identification with weighted scoring
GKG_PORTS_OV_SECOND_APPROACH = (    
    GKG_NOT_EMOTIONAL_CHARGE
    
    # STEP 1: Apply base filters (same as Approach 1)
    .withColumn("BaseNews",
        when(
            (col("THEMES").like("%PORT%")) &          # Must mention ports
            (col("THEMES").like("%TRANSPORT%")) &     # Must mention transport
            (~col("THEMES").like("%AIRPORT%")),       # Exclude airports
            1
        )
    )
    .filter("BaseNews == 1")
    
    # STEP 2: Tag with theme flags
    .withColumn("NewsWithTINFA", when(col("THEMES").like("%TRANSPORT_INFRASTRUCTURE%"), 1))
    .withColumn("NewsWithTRADE", when(col("THEMES").like("%TRADE%"), 1))
    .withColumn("NewsWithME", when(col("THEMES").like("%MACROECONOMIC%"), 1))
    .withColumn("NewsWithPS", when(col("THEMES").like("%PUBLIC_SECTOR%"), 1))
    .withColumn("NewsWithMI", when(col("THEMES").like("%MARITIME_INCIDENT%"), 1))
    .fillna(0)
    
    # STEP 3: Count total themes per article
    .withColumn("Total",
        col("NewsWithTINFA") + col("NewsWithTRADE") + 
        col("NewsWithME") + col("NewsWithPS") + col("NewsWithMI")
    )
    
    # STEP 4: Filter to negative news only
    .filter("AverageTone < 0")
    
    # STEP 5: Aggregate by date, theme count, and location
    .groupby("Date", "Total", "LocationCode")
    .agg(count("Total").alias("NumberOfNews"))
    
    # STEP 6: Apply exponential weighting based on theme count
    # More themes = exponentially higher importance score
    .withColumn('WeightedCountOfNews',
        when(col("Total") == 5, col("NumberOfNews") * 500)   # 5 themes: Critical (500x weight)
        .when(col('Total') == 4, col("NumberOfNews") * 250)  # 4 themes: Major (250x weight)
        .when(col("Total") == 3, col("NumberOfNews") * 100)  # 3 themes: Significant (100x weight)
        .when(col("Total") == 2, col("NumberOfNews") * 5)    # 2 themes: Moderate (5x weight)
        .otherwise(0)  # 1 theme: Filtered out as noise (0 weight)
    )
)

# PART 2: Create complete date × location × theme grid (same as Approach 1)
GKG_PORTS_OV_SECOND_APPROACH_CD = (
    TABLE_OF_DATES
    .crossJoin(
        GKG_PORTS_OV_SECOND_APPROACH
        .select("Total", "LocationCode")
        .distinct()
    )
)

# PART 3: Join and fill missing combinations
GKG_PORTS_SECOND_APPROACH = (
    GKG_PORTS_OV_SECOND_APPROACH_CD
    .join(GKG_PORTS_OV_SECOND_APPROACH, ["Date", "Total", "LocationCode"], "left")
    .fillna(0)  # Fill missing dates with 0
    .persist()  # Cache for visualization
)

# Materialize the dataframe
GKG_PORTS_SECOND_APPROACH.count()

# NOTE: Visualization cells below aggregate weighted scores by location
# and calculate day-over-day changes to detect disruption risk spikes

In [0]:
display(
GKG_PORTS_SECOND_APPROACH
.groupBy("Date","LocationCode").agg(sum("WeightedCountOfNews").alias("WeightedCountOfNews"))
.withColumn("LagWeightedCountNews", lag(col("WeightedCountOfNews"),2).over(Window.partitionBy("LocationCode").orderBy("Date")))
.fillna(0)
.filter(col("LocationCode").isin(
        'CA02',  # Port of Vancouver - British Columbia
        'CA10',  # Port of Montreal - Quebec
        'USCA',  # Port of Los Angeles, Long Beach, Oakland - California
        'CH23',  # Port of Shanghai - Shanghai
        'CH30',  # Port of Shenzhen - Guangdong Province
        'CH02',  # Port of Ningbo-Zhoushan - Zhejiang Province
        'JA40',  # Port of Tokyo - Tokyo
        'JA19',  # Port of Yokohama - Kanagawa Prefecture
        'JA01'  # Port of Nagoya - Aichi Prefecture
    ))
)

Databricks visualization. Run in Databricks to view.

---

## 9. Summary and Key Takeaways

### What We Accomplished:

✅ **Data Quality Assurance**
- Cleaned and validated port location coordinates
- Standardized GDELT GKG data formats  
- Filtered emotionally-charged news to ensure objective analysis

✅ **Advanced Analytics Techniques**
- **Approach 1**: Time-series growth analysis of theme-specific news
- **Approach 2**: Weighted scoring system for multi-theme news prioritization

✅ **Predictive Framework**
- Established baseline metrics for normal news patterns
- Created lagging indicators to detect anomalous spikes
- Built foundation for real-time disruption alerting

### Data Quality Best Practices Demonstrated:

1. **Validation at Source**: Filter NULL values before processing
2. **Standardization**: Convert all coordinates to consistent decimal format
3. **Bias Removal**: Filter emotionally-charged content for objective analysis
4. **Completeness**: Use cross-join to ensure all date/location combinations present
5. **Documentation**: Clear comments explaining every transformation step

### Next Steps:

- 📊 **Visualization**: Create Power BI dashboards for operational monitoring
- 🤖 **ML Models**: Train predictive models on historical patterns
- ⚠️ **Alerting**: Implement real-time alerts when thresholds exceeded
- 📈 **Validation**: Back-test against known disruption events

---

**End of Notebook**